In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd "/content/drive/My Drive/gc-mc-master/gcmc/"

/content/drive/My Drive/gc-mc-master/gcmc


In [ ]:
!python setup.py install
!pip3 install pickle-mixin

In [ ]:
from __future__ import division
from __future__ import print_function

import argparse
import datetime
import time
import random

import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import scipy.sparse as sp
import pandas as pd
import sys
import json

from model import RecommenderGAE, RecommenderSideInfoGAE
from utils import construct_feed_dict

In [ ]:
# Set random seed
# seed = 123 # use only for unit testing
seed = int(time.time())
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
NUMCLASSES = 5
FEATURES = False # 有没有features
SELFCONNECTIONS = False
ACCUM = sum
HIDDEN = [500,75]
DO = 0.7 #dropout

In [ ]:
u_features = None
v_features = None
fname = 'ml_1m'
print('Loading dataset', fname)

data_dir = 'data/' + fname


files = ['/ratings.dat', '/movies.dat', '/users.dat']

sep = r'\:\:'
filename = data_dir + files[0]

dtypes = {
    'u_nodes': np.int64, 'v_nodes': np.int64,
    'ratings': np.float32, 'timestamp': np.float64}

# use engine='python' to ignore warning about switching to python backend when using regexp for sep
data = pd.read_csv(filename, sep=sep, header=None,
                    names=['u_nodes', 'v_nodes', 'ratings', 'timestamp'], converters=dtypes, engine='python')
data

Loading dataset ml_1m


,u_nodes,v_nodes,ratings,timestamp
0,1,1193,5.0,978300760.0
1,1,661,3.0,978302109.0
2,1,914,3.0,978301968.0
3,1,3408,4.0,978300275.0
4,1,2355,5.0,978824291.0
...,...,...,...,...
1000204,6040,1091,1.0,956716541.0
1000205,6040,1094,5.0,956704887.0
1000206,6040,562,5.0,956704746.0
1000207,6040,1096,4.0,956715648.0


In [ ]:
# make sure to convert to list, otherwise random.shuffle acts weird on it without a warning
data_array = data.values.tolist()
random.seed(seed)
random.shuffle(data_array)
data_array = np.array(data_array)

u_nodes_ratings = data_array[:, 0].astype(dtypes['u_nodes'])
v_nodes_ratings = data_array[:, 1].astype(dtypes['v_nodes'])
ratings = data_array[:, 2].astype(dtypes['ratings'])

u_nodes_ratings

array([4989, 1949,  735, ..., 3471, 5954, 2878])

In [ ]:
# Map data to proper indices in case they are not in a continues [0, N) range
def map_data(data):
  uniq = list(set(data))

  id_dict = {old: new for new, old in enumerate(sorted(uniq))}
  data = np.array(list(map(lambda x: id_dict[x], data)))
  n = len(uniq)

  return data, id_dict, n

In [ ]:
num_users = len(list(set(u_nodes_ratings)))
num_items = len(list(set(v_nodes_ratings)))

print('Number of users = %d' % num_users)
print('Number of items = %d' % num_items)

Number of users = 6040
Number of items = 3706


In [ ]:
u_nodes_ratings, u_dict, num_users = map_data(u_nodes_ratings)
v_nodes_ratings, v_dict, num_items = map_data(v_nodes_ratings)

u_nodes_ratings = u_nodes_ratings.astype(np.int64)
v_nodes_ratings = v_nodes_ratings.astype(np.int64)
ratings = ratings.astype(np.float32)

ratings

array([5., 5., 4., ..., 3., 3., 2.], dtype=float32)

In [ ]:
# Load movie features
movies_file = data_dir + files[1]

movies_headers = ['movie_id', 'title', 'genre']
movies_df = pd.read_csv(movies_file, sep=sep, header=None,
                        names=movies_headers, engine='python', encoding= 'ISO-8859-1')

movies_df

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
# Extracting all genres
genres = []
for s in movies_df['genre'].values:
    genres.extend(s.split('|'))

genres = list(set(genres))
num_genres = len(genres)

genres_dict = {g: idx for idx, g in enumerate(genres)}

# Creating 0 or 1 valued features for all genres
v_features = np.zeros((num_items, num_genres), dtype=np.float32)
for movie_id, s in zip(movies_df['movie_id'].values.tolist(), movies_df['genre'].values.tolist()):
    # Check if movie_id was listed in ratings file and therefore in mapping dictionary
    if movie_id in v_dict.keys():
        gen = s.split('|')
        for g in gen:
            v_features[v_dict[movie_id], genres_dict[g]] = 1.

v_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# Load user features
users_file = data_dir + files[2]
users_headers = ['user_id', 'gender', 'age', 'occupation', 'zip-code']
users_df = pd.read_csv(users_file, sep=sep, header=None,
                        names=users_headers, engine='python')

users_df

,user_id,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [ ]:
# Extracting all features
cols = users_df.columns.values[1:]

cntr = 0
feat_dicts = []
for header in cols:
    d = dict()
    feats = np.unique(users_df[header].values).tolist()
    d.update({f: i for i, f in enumerate(feats, start=cntr)})
    feat_dicts.append(d)
    cntr += len(d)

num_feats = sum(len(d) for d in feat_dicts)

u_features = np.zeros((num_users, num_feats), dtype=np.float32)
for _, row in users_df.iterrows():
    u_id = row['user_id']
    if u_id in u_dict.keys():
        for k, header in enumerate(cols):
            u_features[u_dict[u_id], feat_dicts[k][row[header]]] = 1.

u_features = sp.csr_matrix(u_features)
v_features = sp.csr_matrix(v_features)

u_features

<6040x3469 sparse matrix of type '<class 'numpy.float32'>'
	with 24160 stored elements in Compressed Sparse Row format>

In [ ]:

print('Number of users = %d' % num_users)
print('Number of items = %d' % num_items)
print('Number of links = %d' % ratings.shape[0])
print('Fraction of positive links = %.4f' % (float(ratings.shape[0]) / (num_users * num_items),))

# load_data 函数到这里结束

Number of users = 6040
Number of items = 3706
Number of links = 1000209
Fraction of positive links = 0.0447


In [ ]:
# num_users, num_items, u_nodes_ratings, v_nodes_ratings, ratings, u_features, v_features

In [ ]:
# 继续 create_trainvaltest_split 函数
import pickle as pkl
datasplit_path = 'data/' + fname + '/split_seed' + str(fname) + '.pickle'

with open(datasplit_path, 'wb+') as f:
    pkl.dump([num_users, num_items, u_nodes_ratings, v_nodes_ratings, ratings, u_features, v_features], f)

In [ ]:
neutral_rating = -1

rating_dict = {r: i for i, r in enumerate(np.sort(np.unique(ratings)).tolist())}

labels = np.full((num_users, num_items), neutral_rating, dtype=np.int32)
labels[u_nodes_ratings, v_nodes_ratings] = np.array([rating_dict[r] for r in ratings])
labels = labels.reshape([-1])
labels

array([ 4, -1, -1, ..., -1, -1, -1], dtype=int32)

In [ ]:
# number of test and validation edges
num_test = int(np.ceil(ratings.shape[0] * 0.1))
num_val = int(np.ceil(ratings.shape[0] * 0.9 * 0.05))

num_train = ratings.shape[0] - num_val - num_test

pairs_nonzero = np.array([[u, v] for u, v in zip(u_nodes_ratings, v_nodes_ratings)])

idx_nonzero = np.array([u * num_items + v for u, v in pairs_nonzero])

train_idx = idx_nonzero[0:num_train]
val_idx = idx_nonzero[num_train:num_train + num_val]
test_idx = idx_nonzero[num_train + num_val:]

train_pairs_idx = pairs_nonzero[0:num_train]
val_pairs_idx = pairs_nonzero[num_train:num_train + num_val]
test_pairs_idx = pairs_nonzero[num_train + num_val:]

u_test_idx, v_test_idx = test_pairs_idx.transpose()
u_val_idx, v_val_idx = val_pairs_idx.transpose()
u_train_idx, v_train_idx = train_pairs_idx.transpose()

# create labels
train_labels = labels[train_idx]
val_labels = labels[val_idx]
test_labels = labels[test_idx]


#testing==true
u_train_idx = np.hstack([u_train_idx, u_val_idx])
v_train_idx = np.hstack([v_train_idx, v_val_idx])
train_labels = np.hstack([train_labels, val_labels])
# for adjacency matrix construction
train_idx = np.hstack([train_idx, val_idx])

# make training adjacency matrix
rating_mx_train = np.zeros(num_users * num_items, dtype=np.float32)
rating_mx_train[train_idx] = labels[train_idx].astype(np.float32) + 1.
rating_mx_train = sp.csr_matrix(rating_mx_train.reshape(num_users, num_items))

class_values = np.sort(np.unique(ratings))
rating_mx_train
# return 
# u_features, v_features
# rating_mx_train
# train_labels
# u_train_idx, v_train_idx
# val_labels
# u_val_idx, v_val_idx
# test_labels
# u_test_idx, v_test_idx
# class_values

<6040x3706 sparse matrix of type '<class 'numpy.float32'>'
	with 900188 stored elements in Compressed Sparse Row format>

In [ ]:
adj_train = rating_mx_train
train_u_indices = u_train_idx
train_v_indices = v_train_idx
val_u_indices = u_val_idx
val_v_indices = v_val_idx
test_u_indices = u_test_idx
test_v_indices = v_test_idx

num_users, num_items = adj_train.shape
num_side_features = 0

In [ ]:
def preprocess_user_item_features(u_features, v_features):
    """
    Creates one big feature matrix out of user features and item features.
    Stacks item features under the user features.
    """

    zero_csr_u = sp.csr_matrix((u_features.shape[0], v_features.shape[1]), dtype=u_features.dtype)
    zero_csr_v = sp.csr_matrix((v_features.shape[0], u_features.shape[1]), dtype=v_features.dtype)

    u_features = sp.hstack([u_features, zero_csr_u], format='csr')
    v_features = sp.hstack([zero_csr_v, v_features], format='csr')

    return u_features, v_features

In [ ]:
# feature loading assume no Features: Netflix dataset no features

# scipy.sparse.identity(n, dtype='d', format=None) 
u_features = sp.identity(num_users, format='csr')
v_features = sp.identity(num_items, format='csr')

u_features, v_features = preprocess_user_item_features(u_features, v_features)
u_features

<6040x9746 sparse matrix of type '<class 'numpy.float64'>'
	with 6040 stored elements in Compressed Sparse Row format>

In [ ]:
# global normalization
support = []
support_t = []
adj_train_int = sp.csr_matrix(adj_train, dtype=np.int32)

for i in range(NUMCLASSES):
    # build individual binary rating matrices (supports) for each rating
    support_unnormalized = sp.csr_matrix(adj_train_int == i + 1, dtype=np.float32)

    if support_unnormalized.nnz == 0 and DATASET != 'yahoo_music':
        # yahoo music has dataset split with not all ratings types present in training set.
        # this produces empty adjacency matrices for these ratings.
        sys.exit('ERROR: normalized bipartite adjacency matrix has only zero entries!!!!!')

    support_unnormalized_transpose = support_unnormalized.T
    support.append(support_unnormalized)
    support_t.append(support_unnormalized_transpose)



In [ ]:
def globally_normalize_bipartite_adjacency(adjacencies, verbose=False, symmetric=True): 
    """ Globally Normalizes set of bipartite adjacency matrices """

    if verbose:
        print('Symmetrically normalizing bipartite adj')
    # degree_u and degree_v are row and column sums of adj+I

    adj_tot = np.sum(adj for adj in adjacencies)
    degree_u = np.asarray(adj_tot.sum(1)).flatten()
    degree_v = np.asarray(adj_tot.sum(0)).flatten()

    # set zeros to inf to avoid dividing by zero
    degree_u[degree_u == 0.] = np.inf
    degree_v[degree_v == 0.] = np.inf

    degree_u_inv_sqrt = 1. / np.sqrt(degree_u)
    degree_v_inv_sqrt = 1. / np.sqrt(degree_v)
    degree_u_inv_sqrt_mat = sp.diags([degree_u_inv_sqrt], [0])
    degree_v_inv_sqrt_mat = sp.diags([degree_v_inv_sqrt], [0])

    degree_u_inv = degree_u_inv_sqrt_mat.dot(degree_u_inv_sqrt_mat)

    if symmetric:
        adj_norm = [degree_u_inv_sqrt_mat.dot(adj).dot(degree_v_inv_sqrt_mat) for adj in adjacencies]

    else:
        adj_norm = [degree_u_inv.dot(adj) for adj in adjacencies]

    return adj_norm

In [ ]:
support = globally_normalize_bipartite_adjacency(support, symmetric=True)
support_t = globally_normalize_bipartite_adjacency(support_t, symmetric=True)

# SELFCONNECTIONS = False
if SELFCONNECTIONS:
    support.append(sp.identity(u_features.shape[0], format='csr'))
    support_t.append(sp.identity(v_features.shape[0], format='csr'))

num_support = len(support)
support = sp.hstack(support, format='csr')
support_t = sp.hstack(support_t, format='csr')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


In [ ]:
# Collect all user and item nodes for test set
test_u = list(set(test_u_indices))
test_v = list(set(test_v_indices))
test_u_dict = {n: i for i, n in enumerate(test_u)}
test_v_dict = {n: i for i, n in enumerate(test_v)}

test_u_indices = np.array([test_u_dict[o] for o in test_u_indices])
test_v_indices = np.array([test_v_dict[o] for o in test_v_indices])

test_support = support[np.array(test_u)]
test_support_t = support_t[np.array(test_v)]

In [ ]:
# Collect all user and item nodes for validation set
val_u = list(set(val_u_indices))
val_v = list(set(val_v_indices))
val_u_dict = {n: i for i, n in enumerate(val_u)}
val_v_dict = {n: i for i, n in enumerate(val_v)}

val_u_indices = np.array([val_u_dict[o] for o in val_u_indices])
val_v_indices = np.array([val_v_dict[o] for o in val_v_indices])

val_support = support[np.array(val_u)]
val_support_t = support_t[np.array(val_v)]

In [ ]:
# Collect all user and item nodes for train set
train_u = list(set(train_u_indices))
train_v = list(set(train_v_indices))
train_u_dict = {n: i for i, n in enumerate(train_u)}
train_v_dict = {n: i for i, n in enumerate(train_v)}

train_u_indices = np.array([train_u_dict[o] for o in train_u_indices])
train_v_indices = np.array([train_v_dict[o] for o in train_v_indices])

train_support = support[np.array(train_u)]
train_support_t = support_t[np.array(train_v)]

In [ ]:
# no features as side info 

test_u_features_side = None
test_v_features_side = None

val_u_features_side = None
val_v_features_side = None

train_u_features_side = None
train_v_features_side = None


In [ ]:
placeholders = {
    'u_features': tf.sparse_placeholder(tf.float32, shape=np.array(u_features.shape, dtype=np.int64)),
    'v_features': tf.sparse_placeholder(tf.float32, shape=np.array(v_features.shape, dtype=np.int64)),
    'u_features_nonzero': tf.placeholder(tf.int32, shape=()),
    'v_features_nonzero': tf.placeholder(tf.int32, shape=()),
    'labels': tf.placeholder(tf.int32, shape=(None,)),

    'u_features_side': tf.placeholder(tf.float32, shape=(None, num_side_features)),
    'v_features_side': tf.placeholder(tf.float32, shape=(None, num_side_features)),

    'user_indices': tf.placeholder(tf.int32, shape=(None,)),
    'item_indices': tf.placeholder(tf.int32, shape=(None,)),

    'class_values': tf.placeholder(tf.float32, shape=class_values.shape),

    'dropout': tf.placeholder_with_default(0., shape=()),
    'weight_decay': tf.placeholder_with_default(0., shape=()),

    'support': tf.sparse_placeholder(tf.float32, shape=(None, None)),
    'support_t': tf.sparse_placeholder(tf.float32, shape=(None, None)),
}

In [ ]:
# create model
model = RecommenderGAE(placeholders,
                           input_dim=u_features.shape[1],
                           num_classes=NUMCLASSES,
                           num_support=num_support,
                           self_connections=SELFCONNECTIONS,
                           num_basis_functions=2,
                           hidden=HIDDEN,
                           num_users=num_users,
                           num_items=num_items,
                           accum="sum",
                           learning_rate=0.01,
                           logging=True)

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/bilinearmixture_1/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/bilinearmixture_1/GatherV2_1_grad/Reshape:0", shape=(None, 75), dtype=float32), dense_shape=Tensor("gradients/bilinearmixture_1/GatherV2_1_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/bilinearmixture_1/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/bilinearmixture_1/GatherV2_grad/Reshape:0", shape=(None, 75), dtype=float32), dense_shape=Tensor("gradients/bilinearmixture_1/

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


In [ ]:
def sparse_to_tuple(sparse_mx):
    """ change of format for sparse matrix. This format is used
    for the feed_dict where sparse matrices need to be linked to placeholders
    representing sparse matrices. """

    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [ ]:
# Convert sparse placeholders to tuples to construct feed_dict
test_support = sparse_to_tuple(test_support)
test_support_t = sparse_to_tuple(test_support_t)

val_support = sparse_to_tuple(val_support)
val_support_t = sparse_to_tuple(val_support_t)

train_support = sparse_to_tuple(train_support)
train_support_t = sparse_to_tuple(train_support_t)

u_features = sparse_to_tuple(u_features)
v_features = sparse_to_tuple(v_features)
assert u_features[2][1] == v_features[2][1], 'Number of features of users and items must be the same!'

num_features = u_features[2][1]
u_features_nonzero = u_features[1].shape[0]
v_features_nonzero = v_features[1].shape[0]

In [ ]:
# Feed_dicts for validation and test set stay constant over different update steps
train_feed_dict = construct_feed_dict(placeholders, u_features, v_features, u_features_nonzero,
                                      v_features_nonzero, train_support, train_support_t,
                                      train_labels, train_u_indices, train_v_indices, class_values, DO,
                                      train_u_features_side, train_v_features_side)
# No dropout for validation and test runs
val_feed_dict = construct_feed_dict(placeholders, u_features, v_features, u_features_nonzero,
                                    v_features_nonzero, val_support, val_support_t,
                                    val_labels, val_u_indices, val_v_indices, class_values, 0.,
                                    val_u_features_side, val_v_features_side)

test_feed_dict = construct_feed_dict(placeholders, u_features, v_features, u_features_nonzero,
                                     v_features_nonzero, test_support, test_support_t,
                                     test_labels, test_u_indices, test_v_indices, class_values, 0.,
                                     test_u_features_side, test_v_features_side)

In [ ]:
# Collect all variables to be logged into summary
merged_summary = tf.summary.merge_all()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

train_summary_writer = None
val_summary_writer = None

best_val_score = np.inf
best_val_loss = np.inf
best_epoch = 0
wait = 0

In [ ]:
print('Training...')
NB_EPOCH = 3500
TESTING = True
for epoch in range(NB_EPOCH):

    t = time.time()

    # Run single weight update
    # outs = sess.run([model.opt_op, model.loss, model.rmse], feed_dict=train_feed_dict)
    # with exponential moving averages
    outs = sess.run([model.training_op, model.loss, model.rmse], feed_dict=train_feed_dict)

    train_avg_loss = outs[1]
    train_rmse = outs[2]

    val_avg_loss, val_rmse = sess.run([model.loss, model.rmse], feed_dict=val_feed_dict)

    
    print("[*] Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(train_avg_loss),
          "train_rmse=", "{:.5f}".format(train_rmse),
          "val_loss=", "{:.5f}".format(val_avg_loss),
          "val_rmse=", "{:.5f}".format(val_rmse),
          "\t\ttime=", "{:.5f}".format(time.time() - t))

    if val_rmse < best_val_score:
        best_val_score = val_rmse
        best_epoch = epoch


    if epoch % 100 == 0 and epoch > 1000 and not TESTING and False:
        saver = tf.train.Saver()
        save_path = saver.save(sess, "tmp/%s_seed%d.ckpt" % (model.name, DATASEED), global_step=model.global_step)

        # load polyak averages
        variables_to_restore = model.variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        saver.restore(sess, save_path)

        val_avg_loss, val_rmse = sess.run([model.loss, model.rmse], feed_dict=val_feed_dict)

        print('polyak val loss = ', val_avg_loss)
        print('polyak val rmse = ', val_rmse)

        # Load back normal variables
        saver = tf.train.Saver()
        saver.restore(sess, save_path)

Training...
[*] Epoch: 0001 train_loss= 1.60944 train_rmse= 1.25967 val_loss= 1.60929 val_rmse= 1.26016 		time= 3.67094
[*] Epoch: 0002 train_loss= 1.60934 train_rmse= 1.25950 val_loss= 1.60152 val_rmse= 1.24805 		time= 0.93432
[*] Epoch: 0003 train_loss= 1.60542 train_rmse= 1.25347 val_loss= 1.54898 val_rmse= 1.07739 		time= 0.94581
[*] Epoch: 0004 train_loss= 1.55060 train_rmse= 1.14569 val_loss= 3.47731 val_rmse= 1.24752 		time= 0.94110
[*] Epoch: 0005 train_loss= 2.06675 train_rmse= 1.14419 val_loss= 2.20080 val_rmse= 1.15809 		time= 0.92982
[*] Epoch: 0006 train_loss= 1.66097 train_rmse= 1.07061 val_loss= 1.52300 val_rmse= 1.06494 		time= 0.94030
[*] Epoch: 0007 train_loss= 1.54320 train_rmse= 1.13790 val_loss= 1.56176 val_rmse= 1.17870 		time= 0.95239
[*] Epoch: 0008 train_loss= 1.58592 train_rmse= 1.22126 val_loss= 1.58560 val_rmse= 1.22108 		time= 0.93552
[*] Epoch: 0009 train_loss= 1.59706 train_rmse= 1.23946 val_loss= 1.59026 val_rmse= 1.23031 		time= 0.95969
[*] Epoch: 0010 

In [ ]:
# store model including exponential moving averages
saver = tf.train.Saver()
save_path = saver.save(sess, "tmp/%s.ckpt" % model.name, global_step=model.global_step)

print("\nOptimization Finished!")
print('best validation score =', best_val_score, 'at iteration', best_epoch)

# Testing
test_avg_loss, test_rmse = sess.run([model.loss, model.rmse], feed_dict=test_feed_dict)
print('test loss = ', test_avg_loss)
print('test rmse = ', test_rmse)

# restore with polyak averages of parameters
variables_to_restore = model.variable_averages.variables_to_restore()
saver = tf.train.Saver(variables_to_restore)
saver.restore(sess, save_path)

test_avg_loss, test_rmse = sess.run([model.loss, model.rmse], feed_dict=test_feed_dict)
print('polyak test loss = ', test_avg_loss)
print('polyak test rmse = ', test_rmse)


print('\nSETTINGS:\n')
for key, val in sorted(vars(ap.parse_args()).items()):
    print(key, val)

print('global seed = ', seed)

# For parsing results from file
ap = argparse.ArgumentParser()
results = vars(ap.parse_args()).copy()
results.update({'best_val_score': float(best_val_score), 'best_epoch': best_epoch})
print(json.dumps(results))

sess.close()


Optimization Finished!
best validation score = 0.8193378 at iteration 2401
test loss =  1.8989594
test rmse =  1.0108596
INFO:tensorflow:Restoring parameters from tmp/recommendergae.ckpt-3500
polyak test loss =  1.8778872
polyak test rmse =  1.013956

SETTINGS:



NameError: ignored